Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import random
from sklearn.metrics import pairwise
import time

# --- initialization ---
background = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_right = 20
roi_left = 300 

countdown_start = 3  # seconds for countdown
round_active = False
round_start_time = 0
show_result = False
result_start_time = 0
result_duration = 3  # seconds to show previous round result
detect_delay = 1     # delay in seconds after countdown before detecting hand

# --- total scores ---
player_total_score = 0
computer_total_score = 0

def calc_accum_avg(frame,accumulated_weight):
    global background
    if background is None:
        background = frame.copy().astype('float')
        return None
    cv2.accumulateWeighted(frame,background,accumulated_weight)

def segment(frame,threshold_min=25):
    diff = cv2.absdiff(background.astype('uint8'),frame)
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    image, contours, hierarchy = cv2.findContours(thresholded.copy(),
                                                  cv2.RETR_EXTERNAL,
                                                  cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours,key=cv2.contourArea)
        return(thresholded,hand_segment)

def count_fingers(thresholded,hand_segment):
    conv_hull = cv2.convexHull(hand_segment)
    top = tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    
    cx = (left[0] + right[0]) // 2
    cy = (top[1] + bottom[1]) // 2
    
    distance = pairwise.euclidean_distances(
        [[cx, cy]], 
        [[left[0], left[1]], [right[0], right[1]], [top[0], top[1]], [bottom[0], bottom[1]]]
    )[0]

    max_distance = distance.max()
    radius = int(0.85*max_distance)
    circumfrence = (2*np.pi*radius)
    
    circular_roi = np.zeros(thresholded.shape,dtype='uint8')
    cv2.circle(circular_roi,(cx,cy),radius,255,10)
    circular_roi = cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    
    image, contours, hierarchy = cv2.findContours(circular_roi.copy(),
                                                  cv2.RETR_EXTERNAL,
                                                  cv2.CHAIN_APPROX_NONE)
    
    count = 0
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        out_of_wrist = (cy + (cy*0.25)) > (y+h)
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        if out_of_wrist and limit_points:
            count += 1

    if count == 0:
        return 0   # rock
    elif count == 2:
        return 2   # scissors
    elif count == 5:
        return 5   # paper
    else:
        return -1  # ignore other cases

def play_game(player_choice):
    global player_total_score, computer_total_score
    options = {0: "Rock", 2: "Scissors", 5: "Paper"}
    computer_choice = random.choice([0,2,5])
    
    if player_choice == computer_choice:
        result = "Tie!"
    elif (player_choice==0 and computer_choice==2) or \
         (player_choice==2 and computer_choice==5) or \
         (player_choice==5 and computer_choice==0):
        result = "Player wins!"
        player_total_score += 1
    else:
        result = "Computer wins!"
        computer_total_score += 1
    
    return options[player_choice], options[computer_choice], result

cam = cv2.VideoCapture(0)
num_frames = 0
current_result = None
detect_time = 0

while True:
    ret,frame = cam.read()
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(11,11),0)
    
    # --- display total score always ---
    cv2.putText(frame_copy,"Player : Computer",(350,50),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.putText(frame_copy,f"{player_total_score}:{computer_total_score}",(350,90),
                cv2.FONT_HERSHEY_SIMPLEX,0.9,(0,255,0),2)
    
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        cv2.putText(frame_copy,'WAIT: GETTING BACKGROUND',(50,50),
                    cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
    else:
        if show_result:
            elapsed_result = int(time.time() - result_start_time)
            if elapsed_result < result_duration:
                player, computer, result_text = current_result
                cv2.putText(frame_copy,f"Player: {player}",(10,50),
                            cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
                cv2.putText(frame_copy,f"Computer: {computer}",(10,100),
                            cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,255),2)
                cv2.putText(frame_copy,result_text,(10,150),
                            cv2.FONT_HERSHEY_SIMPLEX,1.2,(0,0,255),3)
            else:
                show_result = False
                round_active = False
        
        if not show_result:
            if not round_active:
                round_active = True
                round_start_time = time.time()
                detect_time = None  # reset detect time
            
            elapsed = int(time.time() - round_start_time)
            countdown = countdown_start - elapsed
            
            if countdown > 0:
                cv2.putText(frame_copy,f"Show now: {countdown}",(50,50),
                            cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
            else:
                if detect_time is None:
                    detect_time = time.time()
                elif time.time() - detect_time >= detect_delay:
                    hand = segment(gray)
                    if hand is not None:
                        thresholded, hand_segment = hand
                        cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],
                                         -1,(255,0,0),5)
                        fingers = count_fingers(thresholded,hand_segment)
                        if fingers in [0,2,5]:
                            current_result = play_game(fingers)
                            show_result = True
                            result_start_time = time.time()

    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frames += 1
    cv2.imshow('Rock Paper Scissors', frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()
